In [3]:
def clean_array(substrs):
    # cleans split string substrings into an array of ints
    cleaned_substr = []
    for substr in substrs:
        substr = re.sub(' ', '', substr)
        substr = re.sub('\[', '', substr)
        substr = re.sub('\]', '', substr)
        substr = re.sub(',', '', substr)
        if len(substr) == 0:
            cleaned_substr.append(1000)
        else:
            cleaned_substr.append(int(substr))
    return cleaned_substr

questiondata = pd.read_csv('https://www.inf.ed.ac.uk/teaching/courses/fds/data/project-2021-2022/eedi/question_metadata_task_3_4.csv.gz')
subdata = pd.read_csv('https://www.inf.ed.ac.uk/teaching/courses/fds/data/project-2021-2022/eedi/subject_metadata.csv.gz')

    # changes the string of subjectIds for each row in question data to an array of ints
questiondata['SubjectId'] = questiondata['SubjectId'].apply(lambda subjectlist: clean_array(subjectlist.split(',')))
    # from subject data, takes the names, parentIds and Levels of each subjectId in array for each row
questiondata['SubjectNames'] = questiondata['SubjectId'].apply(lambda subjects : [subdata[subdata['SubjectId'] == subject]['Name'] for subject in subjects])
questiondata['ParentIds'] = questiondata['SubjectId'].apply(lambda subjects : [subdata[subdata['SubjectId'] == subject]['ParentId'] for subject in subjects])
questiondata['Levels'] = questiondata['SubjectId'].apply(lambda subjects : [subdata[subdata['SubjectId'] == subject]['Level'] for subject in subjects])

mastersub = pd.merge(master, questiondata, on = 'QuestionId')
confsub = mastersub[pd.notnull(mastersub['Confidence'])]
confsub.head()
confsubgroup = confsub.groupby('QuestionId').agg('sum').reset_index()
confsubgroup['AnswersForQuestion'] = confsubgroup['IsAnswered']
# add here anything else you want to 
questionstats = confsubgroup[['QuestionId','AnswersForQuestion','IsCorrect','Confidence','Overconfidence']].merge(questiondata, how = 'left')
questionstats.drop(['SubjectNames','ParentIds','Levels'],axis = 1, inplace = True)
# question stats now has the total sum for correctness, confidence and overconfidence

# this is how you would just get level 2 stuff
#level2 = subdata[subdata.Level == 2]
level = subdata.copy()
sl = level.shape[0]
level['Count'] = [0] * sl
level['TotalConf'] = [0] * sl
level['TotalCorrect'] = [0] * sl
level = level.reset_index()

#print(level.head())

for i in range(questionstats.shape[0]):
    for levelsub in level.SubjectId.unique():
        if levelsub in questionstats['SubjectId'][i]:
            subjectIndex = level.index[level['SubjectId'] == levelsub].tolist()[0]
            level['Count'][subjectIndex] = level['Count'][subjectIndex] + questionstats['AnswersForQuestion'][i]
            level['TotalConf'][subjectIndex] = level['TotalConf'][subjectIndex] + questionstats['Confidence'][i]
            level['TotalCorrect'][subjectIndex] = level['TotalCorrect'][subjectIndex] + questionstats['IsCorrect'][i]

level['ConfRatio'] = level['TotalConf'] / level['Count']
level['CorrectRatio'] = level['TotalCorrect'] / level['Count']

level.head()


/tmp/ipykernel_48/2821494482.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  level['Count'][subjectIndex] = level['Count'][subjectIndex] + questionstats['AnswersForQuestion'][i]
/tmp/ipykernel_48/2821494482.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  level['TotalConf'][subjectIndex] = level['TotalConf'][subjectIndex] + questionstats['Confidence'][i]
/tmp/ipykernel_48/2821494482.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

,index,SubjectId,Name,ParentId,Level,Count,TotalConf,TotalCorrect,ConfRatio,CorrectRatio
0,0,3,Maths,NaN,0,346428,25617525,215481,73.947617,0.622008
1,1,32,Number,3.0,1,208815,15458650,133032,74.030362,0.637081
2,2,33,BIDMAS,144.0,3,22593,1694425,13767,74.997787,0.609348
3,3,34,Upper and Lower Bounds,141.0,3,0,0,0,NaN,NaN
4,4,35,Calculator Use,32.0,2,0,0,0,NaN,NaN
